>Explicación teórica de UPGMA:

 >>   Árbol ultramétrico: En un árbol ultramétrico, todas las hojas están a la misma distancia de la raíz. Esto significa que el método UPGMA asume una tasa de evolución constante, lo cual puede no ser realista para todas las especies.

>Actividades y Preguntas

 >>   Otros métodos de cálculo de distancias:Existen métodos como el modelo de Jukes-Cantor y el modelo de Kimura que calculan distancias evolutivas específicas para secuencias de nucleótidos y aminoácidos respectivamente. El modelo de Jukes-Cantor asume que todas las mutaciones tienen la misma probabilidad, mientras que Kimura permite diferentes probabilidades para transiciones y transversiones.

    
   >> Significado de un árbol ultramétrico:Un árbol ultramétrico asume que todas las especies evolucionan a una tasa constante, lo cual implica que la distancia desde cualquier hoja hasta la raíz es la misma.Esto es una simplificación y puede no ser adecuado para linajes con tasas de evolución muy diferentes.

In [16]:
import numpy as np
from collections import defaultdict
import matplotlib.pyplot as plt
import pandas as pd
from scipy.cluster.hierarchy import linkage, dendrogram
import matplotlib.pyplot as plt
# Paso 1: Definir las secuencias.
sequences = {
    'A': "AGTAGTTC",
    'B': "AGTAGTTA",
    'C': "AGTAGTAA",
    'D': "AGTAGGGG",
    'E': "AGTAGGGC"
}

# Inicializar clusters individuales.
clusters = {key: [seq] for key, seq in sequences.items()}
cluster_history =[]
iteracion = 1


# Paso 2: Calcular la matriz de distancias en base a las diferencias entre secuencias.
def calculate_distance_matrix(clusters):
    labels = list(clusters.keys())
    size = len(labels)
    dist_matrix = np.zeros((size, size))
    for i in range(size):
        for j in range(i):
            seqs_i = clusters[labels[i]]
            seqs_j = clusters[labels[j]]
            # Calcular la distancia promedio entre todos los pares de secuencias.
            distances = []
            for seq_i in seqs_i:
                for seq_j in seqs_j:
                    dist = sum(1 for a, b in zip(seq_i, seq_j) if a != b)
                    distances.append(dist)
            avg_distance = np.mean(distances)
            dist_matrix[i][j] = avg_distance
            dist_matrix[j][i] = avg_distance
    return dist_matrix, labels

def plot_dendrogram(cluster_history, labels):
    plt.figure(figsize=(10, 5))
    dendrogram(cluster_history, labels=labels, orientation='top', distance_sort='descending', show_leaf_counts=True)
    plt.title('Dendrograma UPGMA')
    plt.xlabel('Clusters')
    plt.ylabel('Distancia')
    plt.show()


#paso 3 tomar como referencia la distancia mas corta
# Mostrar la matriz de distancias en formato tabular
while len(clusters) > 1:
    print(f"\nIteración {iteracion}:")
    # Calcular la matriz de distancias
    dist_matrix, labels = calculate_distance_matrix(clusters)
    df = pd.DataFrame(dist_matrix, index=labels, columns=labels).round(3)
    print("Matriz de distancias:")
    print(df)
    # Encontrar la distancia mínima y las parejas correspondientes
    min_dist = np.min(dist_matrix[np.nonzero(dist_matrix)])
    pairs = []
    size = len(labels)
    for i in range(size):
        for j in range(i):
            if dist_matrix[i][j] == min_dist:
                pairs.append((labels[i], labels[j]))
                break  # Tomamos solo la primera pareja mínima
        else:
            continue
        break
    # Fusionar los clusters con distancia mínima
    key1, key2 = pairs[0]
    new_key = f"({key1}{key2})"
    merged_sequences = clusters[key1] + clusters[key2]
    # Crear nueva secuencia con gaps donde difieran
    sequence_length = len(merged_sequences[0])
    new_sequence = ''
    for idx in range(sequence_length):
        chars = {seq[idx] for seq in merged_sequences}
        if len(chars) == 1:
            new_sequence += chars.pop()
        else:
            new_sequence += '-'
    # Actualizar clusters
    clusters[new_key] = merged_sequences
    del clusters[key1]
    del clusters[key2]
    print(f"Se unieron {key1} y {key2} en {new_key} con distancia {min_dist}")
    print(f"Nueva secuencia [{new_key}]: {new_sequence}")
    
    
    iteracion += 1

print("\nClúster final:")
final_key = list(clusters.keys())[0]
print(f"Secuencia final {final_key}: {new_sequence}")







'''
Paso 3: Convertir la matriz de distancias a un formato compatible con linkage
condensed_dist_matrix = dist_matrix[np.triu_indices(len(labels), k=1)]

# Paso 4: Utilizar linkage para aplicar UPGMA y crear el dendrograma
linked = linkage(condensed_dist_matrix, method='average')

# Paso 5: Graficar el dendrograma para representar el árbol filogenético
plt.figure(figsize=(10, 7))
dendrogram(linked, labels=labels, distance_sort='descending', show_leaf_counts=True)
plt.title("Árbol Filogenético usando UPGMA", size=15)
plt.xlabel("Secuencias")
plt.ylabel("Distancia")
plt.show()
'''


Iteración 1:
Matriz de distancias:
     A    B    C    D    E
A  0.0  1.0  2.0  3.0  2.0
B  1.0  0.0  1.0  3.0  3.0
C  2.0  1.0  0.0  3.0  3.0
D  3.0  3.0  3.0  0.0  1.0
E  2.0  3.0  3.0  1.0  0.0
Se unieron B y A en (BA) con distancia 1.0
Nueva secuencia [(BA)]: AGTAGTT-

Iteración 2:
Matriz de distancias:
        C    D    E  (BA)
C     0.0  3.0  3.0   1.5
D     3.0  0.0  1.0   3.0
E     3.0  1.0  0.0   2.5
(BA)  1.5  3.0  2.5   0.0
Se unieron E y D en (ED) con distancia 1.0
Nueva secuencia [(ED)]: AGTAGGG-

Iteración 3:
Matriz de distancias:
        C  (BA)  (ED)
C     0.0  1.50  3.00
(BA)  1.5  0.00  2.75
(ED)  3.0  2.75  0.00
Se unieron (BA) y C en ((BA)C) con distancia 1.5
Nueva secuencia [((BA)C)]: AGTAGT--

Iteración 4:
Matriz de distancias:
          (ED)  ((BA)C)
(ED)     0.000    2.833
((BA)C)  2.833    0.000
Se unieron ((BA)C) y (ED) en (((BA)C)(ED)) con distancia 2.8333333333333335
Nueva secuencia [(((BA)C)(ED))]: AGTAG---

Clúster final:
Secuencia final (((BA)C)(ED)): AG

'\nPaso 3: Convertir la matriz de distancias a un formato compatible con linkage\ncondensed_dist_matrix = dist_matrix[np.triu_indices(len(labels), k=1)]\n\n# Paso 4: Utilizar linkage para aplicar UPGMA y crear el dendrograma\nlinked = linkage(condensed_dist_matrix, method=\'average\')\n\n# Paso 5: Graficar el dendrograma para representar el árbol filogenético\nplt.figure(figsize=(10, 7))\ndendrogram(linked, labels=labels, distance_sort=\'descending\', show_leaf_counts=True)\nplt.title("Árbol Filogenético usando UPGMA", size=15)\nplt.xlabel("Secuencias")\nplt.ylabel("Distancia")\nplt.show()\n'